# Comandos do MYSQL



### Para criar tabelas:

**EXEMPLO1**

``` mysql
DROP TABLE IF EXISTS Filmes

CREATE TABLE Filmes(
    id_filme INT NOT NULL AUTO_INCREMENT,,
    titulo VARCHAR(80) NOT NULL,
    PRIMARY KEY (id_filme)
);

CREATE TABLE Filme_Cenas(
    id_filme INT NOT NULL AUTO_INCREMENT,
    id_cena INT NOT NULL,
    PRIMARY KEY(id_filme, id_cena)
);

ALTER TABLE Filme_Cenas
    ADD CONSTRAINT fk_cena
        FOREIGN KEY (id_cena)
            REFERENCES Cena(id_cena);


ALTER TABLE Filme_Cenas
    ADD CONSTRAINT fk_filme
        FOREIGN KEY (id_filme)
            REFERENCES Filmes(id_filme);
            
ALTER TABLE Funcionario
	ADD COLUMN (
		RG_mentor INT NOT NULL,
		orgao_mentor VARCHAR(45) NOT NULL
 );
            
```

**EXEMPLO2**

```mysql
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);

```

# SINTAXE DE JOINS, LEFT , RIGHT E FUNÇÕES QUE APRENDEMOS ANTES DA PI


### Para inserir algo:

In [ ]:
db('''
INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES
    (1, 'Juca', 'Silva'), 
    (2, 'Mario', 'Ferreira'), 
    (3, 'Ana', 'Soares'), 
    (4, 'Antonio', 'Reis'), 
    (5, 'Paulo', 'Oliveira')
''')

### Para ver quais tabelas existem:

In [ ]:
db('SHOW TABLES')

### Para saber qual schema de uma tabela - no caso aqui a CD



In [ ]:
db('DESCRIBE CD')

### SELECT TUDO:

In [ ]:
db('SELECT * FROM CD')

### SELECT nomes que terminem em A (LIKE)

In [ ]:
db("SELECT * FROM CD WHERE Nome_CD LIKE '%a'")

### SELECT com LIKE para detectar se uma string aparece dentro de outra string

In [ ]:
db('''
SELECT
    SUM(minuto_final - minuto_inicio) as minutos_total
FROM 
    Cena
    INNER JOIN Figurino USING (id_cena)
    INNER JOIN Roupa USING (id_roupa)
WHERE
    Roupa.descricao LIKE "%jeans%"
''')

### SELECT COM LIKE PARA VER O QUE COMEÇA COM Rap

In [ ]:
db("SELECT * FROM Jogadores WHERE nome_equipe LIKE 'Rap%'")

### Diferentes SELECT's com comandos importantes:

#### SELECT com True:

In [ ]:
db('SELECT * FROM CD WHERE True')

#### SELECT com DISTINCT (pega só os CDs distintos, não repete):

In [ ]:
db('''
SELECT DISTINCT 
    CD_Indicado 
FROM 
    CD
''')

#### SELECT com COUNT - total de músicas na base de dados



In [ ]:
db('SELECT COUNT(*) FROM MUSICA')

### SELECT com MAX:

In [ ]:
db('SELECT MAX(duracao) FROM MUSICA')

### Criação de variável:

In [ ]:
db('SELECT MAX(duracao) INTO @max_duracao from MUSICA')

### SELECT com AVG:

In [ ]:
db('SELECT AVG(duracao) INTO @media_duracao from MUSICA')

### LEFT

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme!

The LEFT() function extracts a number of characters from a string (starting from left).

In [ ]:
# Primeiro: vamos ver os atores daquele filme. Group por filme
# Depois vamos ver as letras do filme e os atores. 

db('''
SELECT 
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    LEFT(title, 1) = LEFT(first_name, 1)    
GROUP BY
    film_id
HAVING
    COUNT(actor_id) > 2
''')

### CONCAT

In [ ]:
# Observe também o uso da função CONCAT(), só para ficar mais bonito!
# É um inner join só que da tabela com ela mesma :D
db('''
SELECT
    CONCAT('"', c1.Nome_CD, '" indicado por "', c2.Nome_CD, '"')
FROM
    CD c1,
    CD c2
WHERE
    c1.Codigo_CD = c2.CD_Indicado
''')

# OUTPUT

# Executando query:
# ('"Perfil" indicado por "Mais do Mesmo"',)
# ('"Elis Regina - Essa Mulher" indicado por "Bate-Boca"',)
# ('"Mais do Mesmo" indicado por "Elis Regina - Essa Mulher"',)
# ('"Mais do Mesmo" indicado por "A Força que nunca Seca"',)
# ('"Bate-Boca" indicado por "Perfil"',)
# ('"Listen Without Prejudice" indicado por "Barry Manilow Greatest Hits Vol I"',)

### GROUP CONCAT

8) Gere uma tabela contendo, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [ ]:
db('DROP TABLE IF EXISTS films_por_categoria')

db('''
CREATE TEMPORARY TABLE films_por_categoria
    SELECT
        actor_id, first_name, last_name, CONCAT(name, ": ", GROUP_CONCAT(title SEPARATOR ", ")) as films 
    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
        INNER JOIN film USING (film_id)
        INNER JOIN film_category USING (film_id)
        INNER JOIN category USING (category_id)
    GROUP BY
        actor_id, category_id
''')

# OUTPUT (retornava filmes só de um genero, quero todos):
#(1, 'PENELOPE', 'GUINESS', 'Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT')
#(3, 'ED', 'CHASE', 'Documentary: ARMY FLINTSTONES, FRENCH HOLIDAY, HALLOWEEN NUTS, HUNTER ALTER, WEDDING APOLLO, YOUNG LANGUAGE')

In [ ]:
# Aqui agrupo por ator e pego TODOS os genero: filmes daquele ator

db('''
SELECT
    MIN(first_name), last_name, GROUP_CONCAT(films SEPARATOR "; ") as all_films 
FROM
    films_por_categoria
GROUP BY
    actor_id

''')

# OUTPUT

# ('PENELOPE', 'GUINESS', 'Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, 
#  WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP;
#  Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN;
#  Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER')


### SEM INNER JOIN

In [ ]:
db('''
SELECT 
    * 
FROM 
    CD as c, GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
''')

### INNER JOIN

In [ ]:
# Quandos as chaves não tem o mesmo nome nas tabelas

db("""
SELECT 
    MUSICA.Nome_Musica
FROM 
    AUTOR
    INNER JOIN MUSICA_AUTOR ON AUTOR.Codigo_Autor = MUSICA_AUTOR.Codigo_Autor
    INNER JOIN MUSICA ON MUSICA_AUTOR.Codigo_Musica = MUSICA.Codigo_Musica
WHERE
    AUTOR.Nome_Autor = 'Renato Russo'
""")


In [ ]:
# Quando as chaves tem o mesmo nome nas tabelas

db("""
SELECT 
    MUSICA.Nome_Musica
FROM 
    AUTOR
    INNER JOIN MUSICA_AUTOR USING (Codigo_Autor)
    INNER JOIN MUSICA USING (Codigo_Musica)
WHERE
    AUTOR.Nome_Autor = 'Renato Russo'
""")


In [ ]:
db('''
SELECT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    first_name = 'Dan'
    AND last_name = 'Harris'
''')

### LEFT OUTER JOIN - exemplo da tabela de comidas e perigos

In [ ]:
#Quero listar todas as comidas com ou sem perigo, e para aquelas que tem perigo quero o nome associado

db('''
SELECT 
    comida.Nome, 
    perigo.Nome 
FROM
    comida
    LEFT OUTER JOIN perigo ON comida.idPerigo = perigo.id
''')

#LEFT OUUTER JOIN: O lado esquerdo vai aparecer integralmente, no caso comida. No lado direito aparece nulo ou as conexões.

In [ ]:
# Quais linguas não tem nenhum filme na locadora

db('''
SELECT
    title, name
FROM
    language
    LEFT OUTER JOIN film USING(language_id)
WHERE
    title is NULL
''')

### RIGHT OUTER JOIN

In [ ]:
#  quisermos fazer o contrário: queremos que todos os perigos apareçam, mesmo que não exista comida associada

db('''
SELECT 
    comida.Nome,
    perigo.Nome
FROM
    comida
    RIGHT OUTER JOIN perigo ON comida.idPerigo = perigo.id
''')

# Agora aparece todos do lado direito, ou seja, os perigos.

In [ ]:
#Liste os perigos que não estão associados a nenhuma comida.

db('''
SELECT 
    perigo.Nome
FROM
    comida
    RIGHT OUTER JOIN perigo ON comida.idPerigo = perigo.id
WHERE 
    comida.idPerigo IS NULL
''')

## OUUUUUUU

db('''
SELECT 
    perigo.Nome
FROM
    perigo
    LEFT OUTER JOIN comida ON comida.idPerigo = perigo.id
WHERE 
    comida.idPerigo IS NULL
''')

### GROUP BY e HAVING

OBS: toda vez que faço group by tenho que agregar com algo no select : exemplo - SUM, MIN, MAX, COUNT wherever.

E só posso usar HAVING depois de ter o GROUP BY

2) Qual a diferença entre WHERE e HAVING?

O HAVING é bem parecido com o WHERE, porém ele é aplicado apenas para grupos como um todo (isto é, para **as colunas que também aparecem na clásula GROUP BY** ou em uma função agregada - o que representa grupos), enquanto o **WHERE é aplicado para linhas individuais de uma tabela**. Importante resssaltar que uma query pode ter tanto o termo WHERE quanto o termo HAVING. Nesse caso, primeiro é aplicado o WHERE para as linhas individuais nas tabelas, que resultam em linhas que cumprem com as condições indicadas ali. Depois o HAVING será aplicado para as linhas no conjunto resultante, de tal forma que apenas grupos que atendam as condições estabelecidas no HAVING irão aparecer no query de saída

In [ ]:
# Quais atores não compartilham seu sobrenome com nenhum outro ator?

db('''
SELECT
    MIN(first_name), MIN(last_name), COUNT(last_name) as unico_sobrenome
FROM
    actor
GROUP BY
    last_name
HAVING
    unico_sobrenome = 1
ORDER BY
    last_name
''')

### VIEW

In [ ]:
#Quantas vezes foram alugados mesmo para filmes que nunca foram alugados: com cnt igual a 0
#Todos os filmes serão presentes mesmo que não haja [cópia física em lugar nenhum = inventory]
# e mesmo que nunca tenha sido alugado na vida
#pode ter quantos nulos for, ele conta como 0
db('''
DROP VIEW IF EXISTS movie_count;
''')

db('''
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
''')

### TEMPORARY TABLE

In [ ]:
db('''DROP TABLE IF EXISTS max_duration''')

db('''
CREATE TEMPORARY TABLE max_duration
    SELECT
        MAX(length) as duration, name, category_id
    FROM
        film
        INNER JOIN film_category USING (film_id)
        INNER JOIN category USING (category_id)
    GROUP BY
        category_id
    ORDER BY
        duration DESC

''')

### SUBQUERY e IN

In [ ]:
# Filmes dos 3 atores mais populares

db('''
SELECT
    first_name, last_name, title
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE 
    film.film_id IN
    (SELECT * FROM (SELECT
            film_id
        FROM
            film
            INNER JOIN inventory USING (film_id)
            INNER JOIN rental USING (inventory_id)
            INNER JOIN payment USING (rental_id)
        GROUP BY
            film_id
        ORDER BY
            SUM(amount) DESC
        LIMIT 3
        ) as SUBQUERY
    )

''')

### UNION

In [ ]:
#Juntar nome e sobrenome de funcionários e clientes

# Nomes dos funcionários
db('DROP TABLE IF EXISTS nomes_staff')

db('''
CREATE TEMPORARY TABLE nomes_staff
    SELECT first_name, last_name FROM staff
''')

# Nomes dos clientes

db('DROP TABLE IF EXISTS nomes_clientes')
db('''
CREATE TEMPORARY TABLE nomes_clientes 
    SELECT first_name, last_name FROM customer
''')

# junta tudo

db('DROP TABLE IF EXISTS nomes_all')

db('''
CREATE TEMPORARY TABLE nomes_all
    (SELECT * FROM nomes_staff) UNION (SELECT * FROM nomes_clientes)
''')

db('SELECT * FROM nomes_all ')

In [ ]:
# Juntar nome e sobrenome de funcionários e clientes usando UNION e subquery

db('''
    (SELECT * FROM (SELECT first_name, last_name FROM customer) as sub1) UNION
    (SELECT * FROM (SELECT first_name, last_name FROM staff) as sub2)
''')


### Exemplos de queries:

In [ ]:
db('''
SELECT 
    Nome_CD 
FROM 
    CD
WHERE 
    Preco_venda >= 13 
ORDER BY 
    Data_Lancamento ASC [ou pode user DESC] 
LIMIT 1
''')

In [ ]:
db('''
SELECT 
    Nome_Gravadora 
FROM 
    GRAVADORA
WHERE 
    Endereco IS NULL
''')

In [ ]:
#Quais cds foram lançados na década de 90 e custam 10 reais ou menos?
# Poderia usar o BETWEEN, ele é inclusivo, os valores de início e final são incluídos

# YEAR(Data_Lancamento) BETWEEN 1990 AND 1999
db('''
SELECT 
    Nome_CD 
FROM 
    CD
WHERE 
    YEAR(Data_Lancamento) >= 1990 AND YEAR(Data_Lancamento) < 2000 AND Preco_venda <= 10 
''')

In [ ]:
db('''
SELECT
    actor_id, first_name, last_name, COUNT(actor_id) as aparicoes
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN film USING (film_id)
GROUP BY
    actor_id
ORDER BY
    aparicoes DESC
LIMIT
    10
''')

# ------------------------------------------------------------------------------------------------------

# SINTAXE DO QUE APREDEMOS DEPOIS DA PI: TRANSAÇÃO, TRIGGER, PROCEDURE, ETC

### Para iniciar uma transação: 

In [ ]:
db('START TRANSACTION')
db('INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES (8, "Carlos", "Sainz")')
db('COMMIT')

# Posso reverter a transação em uma outra transação

db('START TRANSACTION')
db('''
SELECT id_usuario INTO @id_usuario 
    FROM usuario 
    WHERE nome = 'Barack' AND sobrenome = 'Trump' 
    ORDER BY id_usuario DESC LIMIT 1;
''')
db('SELECT @id_usuario')
db('DELETE FROM usuario WHERE id_usuario=@id_usuario')
db('COMMIT')

### Para criar funções para transação seguir o padrão:



In [ ]:
def deposito(connection, id_usuario, valor):
    with connection.cursor() as cursor:
        # Atualizei o saldo da pessoa depois do depósito
        query1 = '''
        UPDATE 
            usuario
        SET
            saldo = saldo + %s 
        WHERE
            id_usuario = %s
        '''
        params1 = (valor, id_usuario)
        cursor.execute(query1, params1)
        
        # Log da movimentação de conta corrente.
        
        query2 = '''
        INSERT INTO movimentacao (id_usuario, valor) VALUES (%s, %s)
        '''
        params2 = (id_usuario, valor)
        
        cursor.execute(query2, params2)

In [ ]:
def pega_id_usuario(connection, nome, sobrenome):
    with connection.cursor() as cursor:
        cursor.execute(
            '''
            SELECT id_usuario 
            FROM usuario 
            WHERE nome = %s AND sobrenome = %s''', (nome, sobrenome))

        results = cursor.fetchall()
        
    if results is None:
        raise KeyError(f'Usuario {nome} {sobrenome} não encontrado.')
    
    return results[0][0]


# retorna último id inserido
cursor.execute('SELECT LAST_INSERT_ID()')


In [ ]:
def atualiza_emprestimo(connection, id_emprestimo, valor):
    with connection.cursor() as cursor:
        # Atualiza saldo do emprestimo.
        query = '''
        UPDATE 
            emprestimo
        SET 
            valor_atual = valor_atual + %s
        WHERE 
            id_emprestimo = %s
        '''
        params = (valor, id_emprestimo)
        cursor.execute(query, params)

        # Log na tabela de operações.
        query = '''
        INSERT INTO operacao (id_emprestimo, valor) VALUES (%s, %s)
        '''
        params = (id_emprestimo, valor)
        cursor.execute(query, params)

### Para testar funções de transação:
    

In [ ]:
dados = [
    ("Ana", "Soares", "Juca", "Silva", 1000),
    ("Ana", "Soares", "Antonio", "Reis", 2000),
    ("Paulo", "Oliveira", "Juca", "Silva", 3000),
]

for nome_credor, sobrenome_credor, \
    nome_devedor, sobrenome_devedor, \
    valor in dados:
    try:
        start_transaction(connection)
        id_credor = pega_id_usuario(
            connection,
            nome_credor,
            sobrenome_credor,
        )
        id_devedor = pega_id_usuario(
            connection,
            nome_devedor,
            sobrenome_devedor,
        )
        id_emprestimo = cria_emprestimo_(connection, id_credor, id_devedor)
        saque(connection, id_credor, valor)
        deposito(connection, id_devedor, valor)
        atualiza_emprestimo(connection, id_emprestimo, valor)
        commit(connection)
    except Exception as e:
        print(e)
        rollback(connection)

### Para criar uma procedure:

``` mysql

USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;

DELIMITER //
CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome) VALUES (novo_nome, novo_sobrenome);
END//
DELIMITER ;

```

``` mysql

USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

DELIMITER //
CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30,2), IN id_usuario_procurado INT)
BEGIN
    DECLARE taxa_cobrada DECIMAL(30,2);
    SELECT IF(saldo(id_usuario_procurado)>0.0, taxa, 0.0) INTO taxa_cobrada;
    UPDATE usuario SET saldo = saldo - taxa WHERE id_usuario = id_usuario_procurado;
END//
DELIMITER ;

```

Note o uso destes comandos DELIMITER. Coisas de MySQL: ele não entende que os ponto-e-virgula internos ao procedimento não sinalizam o final do comando CREATE PROCEDURE...

### Como chamar e usar procedures: CALL.

Observação: prefira trabalhar com isso no Workbench, é melhor. Já teve bug uma vez.

In [ ]:
db('START TRANSACTION')
try:
    db("CALL adiciona_usuario('Max', 'Verstappen');")
    db("CALL adiciona_usuario('Lando', 'Norris');")
    db("CALL adiciona_usuario('Charles', 'Leclerc');")
except Exception as e:
    print(e)
    db('ROLLBACK');

- Para criar uma function:

``` mysql
USE emprestimos;

DROP FUNCTION IF EXISTS saldo;

DELIMITER //
CREATE FUNCTION saldo(id INT) RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE saldo_procurado DECIMAL(30, 2);
    SELECT IFNULL(saldo, 0.0) INTO saldo_procurado FROM usuario WHERE id_usuario = id;
    RETURN saldo_procurado;
END//
DELIMITER ;

COMMIT;
```


``` mysql

USE emprestimos;

DROP FUNCTION IF EXISTS total_saldo;

DELIMITER //
CREATE FUNCTION total_saldo() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE soma_total DECIMAL(30, 2);
    SELECT 
        SUM(saldo) INTO soma_total 
    FROM usuario;
    RETURN soma_total;
END//
DELIMITER ;

```

### Para usar uma função e chamá-la:

In [ ]:
db("SELECT saldo(1)")

OBS2: Prefira rodar no MySql Workbench os testes. Caso esteja "hanging", simplesmente feche a conexão do notebook e espere rodar tudo no Workbench. Depois abra de novo o jupyter e siga a vida :)

### Funções do MySQL:

https://dev.mysql.com/doc/refman/8.0/en/flow-control-functions.html#function_if

- IF:

```mysql
mysql> SELECT IF(1>2,2,3);
        -> 3
mysql> SELECT IF(1<2,'yes','no');
        -> 'yes'
mysql> SELECT IF(STRCMP('test','test1'),'no','yes');
        -> 'no'
```

- IFNULL:


``` mysql

mysql> SELECT IFNULL(1,0);
        -> 1
mysql> SELECT IFNULL(NULL,10);
        -> 10
mysql> SELECT IFNULL(1/0,10);
        -> 10
mysql> SELECT IFNULL(1/0,'yes');
        -> 'yes'
```

### Para criar um trigger:

Trigger simples que toda vez que tenho uma movimentação atualiza o saldo do usuário. Ou seja, posta sempre o conteúdo na movimentação:

``` mysql

USE emprestimos;

DROP TRIGGER IF EXISTS trig_movimentacao;

DELIMITER //
CREATE TRIGGER trig_movimentacao 
BEFORE INSERT ON movimentacao
FOR EACH ROW
BEGIN
    UPDATE usuario 
        SET saldo = saldo + NEW.valor 
        WHERE id_usuario = NEW.id_usuario;
END//

DELIMITER ;

```

OBS: Se for DELETE, precisa usar o .OLD

Nesse contexto a operação é basicamente: já existe um empréstimo, vou aumentar mais. Ou seja, estou criando uma operação que vai agir atualizando o empréstimo que já existe e criando movimentações. E como a movimentação já tem o trigger anterior, também atualiza o saldo do usuário.


``` mysql

USE emprestimos;

DROP TRIGGER IF EXISTS trig_operacao;

DELIMITER //
CREATE TRIGGER trig_operacao
BEFORE INSERT ON operacao
FOR EACH ROW
BEGIN
	UPDATE emprestimo 
		SET valor_atual = valor_atual + NEW.valor # valor da operacao - aumentando a dívida. Precisamos inserir mov 
        WHERE id_emprestimo = NEW.id_emprestimo;
	SELECT id_credor INTO @id_credor
		FROM emprestimo 
		WHERE id_emprestimo = NEW.id_emprestimo;
	SELECT id_devedor INTO @id_devedor
		FROM emprestimo 
		WHERE id_emprestimo = NEW.id_emprestimo;
	INSERT INTO movimentacao (id_usuario, valor) VALUES (@id_credor, -NEW.valor);
	INSERT INTO movimentacao (id_usuario, valor) VALUES (@id_devedor, NEW.valor);
END //

DELIMITER ;

```


### Para testar o Trigger:

```mysql
USE emprestimos;

SELECT * FROM usuario;
SELECT * FROM emprestimo;
SELECT * FROM operacao;

START TRANSACTION;

INSERT INTO movimentacao (id_usuario, valor) VALUES(1, 50000);

INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 2);
INSERT INTO operacao (id_emprestimo, valor) VALUES (LAST_INSERT_ID(), 2000);

SELECT * FROM usuario;
SELECT * FROM emprestimo;
SELECT * FROM operacao;

ROLLBACK;

SELECT * FROM usuario;
SELECT * FROM emprestimo;
SELECT * FROM operacao;

```

### Para adicionar uma constraint de verificação de valor:

CONSTRAINT c_saldo CHECK (saldo >= 0.0)

``` mysql


CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

```

### Para criar uma VIEW:

Lista nome e sobrenome de usuários sem revelar saldos: 
```mysql
USE emprestimos;

CREATE VIEW nomes AS 
    SELECT DISTINCT nome, sobrenome FROM usuario;
```

Porque usar views? Assim como no caso de stored procedures, podemos configurar permissões de acesso diferentes para esta view. Suponha que um vendedor deva ter acesso aos nomes dos clientes, mas não aos seus saldos (por razões de confidencialidade). Podemos conceder ao vendedor acesso apenas à essa view. Poderíamos ter resolvido o problema também com uma stored procedure: em SQL as coisas costumam ter várias soluções possíveis...

```mySQL

USE emprestimos

CREATE VIEW credor AS
    SELECT 
        id_usuario, SUM(IFNULL(valor_atual, 0.0)) as total
    FROM 
        usuario
        LEFT OUTER JOIN emprestimo ON usuario.id_usuario = emprestimo.id_credor
    GROUP BY
        id_usuario
```

```mySQL

USE emprestimos

CREATE VIEW devedor AS
    SELECT 
        id_usuario, SUM(IFNULL(valor_atual, 0.0)) as total
    FROM 
        usuario
        LEFT OUTER JOIN emprestimo ON usuario.id_usuario = emprestimo.id_devedor
    GROUP BY
        id_usuario
```
```

```mySQL

USE emprestimos

CREATE VIEW valor_liquido AS
    SELECT 
        id_usuario, nome, sobrenome, saldo + credor.total - devedor.total as valor
    FROM 
        usuario
        INNER JOIN credor USING (id_usuario)
        INNER JOIN devedor USING (id_usuario)
```

# ------------------------------------------------------------------------------------------------------

# OPERAÇÕES COM LAMBDA NO PYTHON:

Escreva uma função em Python usando apenas ferramentas de programação funcional (como map(), functools.reduce(), filter(), zip(), list comprehensions, etc.) para a seguinte tarefa:

São dadas duas listas x e y de números reais, de mesmo comprimento n. Cada elemento de cada lista é um número real ou None, indicando que o número está ausente.
A função deverá calcular a soma a seguir:

- somatório de (xi-yi)^2, com i indo de 1 até n e xi != None E yi != None

In [ ]:
# Filtrando individualmente :

x = [10, 5, 6, 7, 9, -6, None, None, 4, 0]
y = [8, 6, None, 5, None, 2, 16, 4, -5, 7]

filtrado_x = list(filter(lambda x : x != None, x))
filtrado_y = list(filter(lambda y: y != None, y))

zippado = zip(filtrado_x, filtrado_y)

print(list(filtrado_x))
print(list(filtrado_y))
print(list(zippado))

somatorio = sum([(a - b)**2 for a, b in zip(filtrado_x, filtrado_y)])

print(somatorio)



In [ ]:
from functools import *

x = [3, None, 5, -2, None]
y = [5, None, None, 1, 3]

zipp_x_y = zip(x, y)

filtro_zip_x_y = list(filter(lambda x : x[0] != None and x[1] != None, zipp_x_y))

print(filtro_zip_x_y)

#somatorio = sum([(a - b)**2 for a, b in filtro_zip_x_y])
#somatorio = reduce(lambda x,y : x+y , (map(lambda x: (x[0] - x[1])**2, filtro_zip_x_y)), 0)
somatorio = sum(map(lambda x: (x[0] - x[1])**2, filtro_zip_x_y))

print(somatorio)